In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import cv2

In [1]:
import glob

In [3]:
# batch_size = 256

In [40]:
def conv_net(data, variable_scope, reuse, name_appendix):
    with tf.variable_scope(variable_scope, reuse=reuse):
        conv1 = tf.layers.conv2d(
            image_left, filters=96, kernel_size=11,
            strides=4, padding='VALID', activation=tf.nn.relu,
            name='conv1')
        conv1 = tf.layers.max_pooling2d(conv1, pool_size=3, strides=2,
                                       padding='VALID')
        conv1 = tf.nn.local_response_normalization(conv1, depth_radius=5, bias=1,
                                                  alpha=0.0001, beta=0.75)
        conv2 = tf.layers.conv2d(conv1, filters=256, kernel_size=5,
                                strides=1, padding='SAME', activation=tf.nn.relu,
                                name='conv2')
        conv2 = tf.layers.max_pooling2d(conv2, pool_size=3, strides=2,
                                       padding='VALID')
        conv2 = tf.nn.local_response_normalization(conv2, depth_radius=5,
                                                  bias=1, alpha=0.0001, beta=0.75)
        conv3 = tf.layers.conv2d(conv2, filters=384, kernel_size=3,
                                strides=1, padding='SAME', activation=tf.nn.relu,
                                name='conv3')
    conv4 = tf.layers.conv2d(conv3, filters=64, kernel_size=1,
                            strides=1, padding='VALID', activation=tf.nn.relu,
                            name='conv4'+'_'+name_appendix)
    conv4 = tf.contrib.layers.flatten(conv4)
    return conv4

In [41]:
def facegrid_net(data):
    flatten = tf.contrib.layers.flatten(data)
    fg_fc1 = tf.layers.dense(
        flatten, 256, activation=tf.nn.relu,                            
        kernel_initializer=tf.truncated_normal_initializer(mean=0, stddev=0.005),
        bias_initializer=tf.constant_initializer(1),
        name='fg_fc1'
    )
    fg_fc2 = tf.layers.dense(fg_fc1, 128, activation=tf.nn.relu,
                            name='fg_fc2')
    return fg_fc2

In [43]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    # Add multiple inputs
    image_left = tf.placeholder(tf.float32, (None, 224, 224, 3), name='image_left') # Channel first or last?
    image_right = tf.placeholder(tf.float32, (None, 224, 224, 3), name='image_right')
    image_face = tf.placeholder(tf.float32, (None, 224, 224, 3), name='image_face')
    facegrid = tf.placeholder(tf.float32, (None, 1, 1, 625), name='facegrid')
    
    # Eyes
    conv4_left = conv_net(image_left, 'eye_conv_net', reuse=False,
                         name_appendix='l')
    conv4_right = conv_net(image_right, 'eye_conv_net', reuse=True,
                          name_appendix='r')
    concat1 = tf.concat([conv4_left, conv4_right], 1)
    fc1 = tf.layers.dense(concat1, 128, activation=tf.nn.relu,
                         name='fc1')
    
    # Face
    conv4_face = conv_net(image_face, 'face_conv_net', reuse=None,
                         name_appendix='f')
    # Flatten first?
    fc1_f = tf.layers.dense(conv4_face, 128, activation=tf.nn.relu,
                            name='fc1_f')
    fc2_f = tf.layers.dense(fc1_f, 64, activation=tf.nn.relu,
                            name='fc2_f')
    
    # FaceGrid
    fg_fc2 = facegrid_net(facegrid)
    
    concat2 = tf.concat([fc1, fg_fc2, fc2_f], axis=1)
    fc2 = tf.layers.dense(concat2, 128, activation=tf.nn.relu,
                         name='fc2')
    fc3 = tf.layers.dense(fc2, 2, name='fc3')

In [44]:
section_name_map = {'l': 'eye_conv_net',
                    'r': 'eye_conv_net',
                    'f': 'face_conv_net'}


def load_external_variables(directory_path):
    # conv{x}_{y} -> different mapping
    # read all file names
    
    file_name = '' # without .npy
    
def file_2_variable(file_name):
    name_parts = file_name.strip().split('_')
    if int(name_parts.pop()) == 0:
        variable_type = 'kernel'
    else:
        variable_type = 'bias'
    if len(name_parts) == 2:
        # section name could be 'f', 'l', 'r', 'fg' 
        # with 'fg', the order is reversed
        layer_type, section_name = name_part
        if layer_type = 'fg':
            layer_type = name_part.join('_')
        variable_scope = section_name_map.get(section_name)
        if variable_scope is not None:
            new_name = '{}/{}/{}:0'.format(variable_scope,
                                          layer_type,
                                          variable_type)
        else:
            new_name = '{}/{}:0'.format(layer_type,
                                        variable_type)
    else:
        new_name = '{}/{}:0'.format(name_part[0],
                                   variable_type)
    

In [45]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #print([var.name for var in tf.global_variables()])

['eye_conv_net/conv1/kernel:0', 'eye_conv_net/conv1/bias:0', 'eye_conv_net/conv2/kernel:0', 'eye_conv_net/conv2/bias:0', 'eye_conv_net/conv3/kernel:0', 'eye_conv_net/conv3/bias:0', 'conv4_l/kernel:0', 'conv4_l/bias:0', 'conv4_r/kernel:0', 'conv4_r/bias:0', 'fc1/kernel:0', 'fc1/bias:0', 'face_conv_net/conv1/kernel:0', 'face_conv_net/conv1/bias:0', 'face_conv_net/conv2/kernel:0', 'face_conv_net/conv2/bias:0', 'face_conv_net/conv3/kernel:0', 'face_conv_net/conv3/bias:0', 'conv4_f/kernel:0', 'conv4_f/bias:0', 'fc1_f/kernel:0', 'fc1_f/bias:0', 'fc2_f/kernel:0', 'fc2_f/bias:0', 'fg_fc1/kernel:0', 'fg_fc1/bias:0', 'fg_fc2/kernel:0', 'fg_fc2/bias:0', 'fc2/kernel:0', 'fc2/bias:0', 'fc3/kernel:0', 'fc3/bias:0']


In [ ]:
tf.get_variable('')

In [ ]:
def predict():
    pass